In [2]:
import pandas as pd
import numpy as np
import os

In [3]:
%pip install pandas openpyxl

In [5]:
directory = '/gdrive/MyDrive/IomT/unsw_data/unsw_raw_data'  # Replace with your directory path
all_files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]

In [6]:
list_of_dataframes = [pd.read_csv(os.path.join(directory, f)) for f in all_files]
merged_dataframe = pd.concat(list_of_dataframes, ignore_index=True)

<ipython-input-6-d7b0382a8dad>:1: DtypeWarning: Columns (24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  list_of_dataframes = [pd.read_csv(os.path.join(directory, f)) for f in all_files]


In [ ]:
# Set Pandas to display all columns and rows
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

In [7]:
df_merged = merged_dataframe

In [8]:
df_merged.tail()

,idx,protocol,src_ip,src_port,dst_ip,dst_port,packet_length,tcp_flags,window_size,ttl,...,Start_Time,End_Time,Duration,Flow_Volume,Flow_Rate,Inter_Arrival_Time,Total_Fwd_Packets,Total_Bwd_Packets,Fwd_Packet_Length_Total,Bwd_Packet_Length_Total
8288305,607198,TCP,192.168.1.106,47940.0,52.87.241.159,443.0,156.0,ack,2549.0,64.0,...,1.475330e+09,1.475413e+09,82797.735474,17729288,214.127692,0.815169,78173,78116,12386218,5343070
8288306,607199,TCP,52.1.40.122,443.0,192.168.1.196,2179.0,54.0,ack,21976.0,32.0,...,1.475330e+09,1.475413e+09,82790.982727,1524540,18.414324,0.215737,11053,10783,799290,725250
8288307,607200,TCP,52.87.241.159,443.0,192.168.1.106,47940.0,66.0,ack,836.0,224.0,...,1.475330e+09,1.475413e+09,82797.735474,17729288,214.127692,0.215153,78173,78116,12386218,5343070
8288308,607201,NaN,fe80::16cc:20ff:fe51:33ea,NaN,ff02::2,NaN,NaN,NaN,NaN,1.0,...,0.000000e+00,0.000000e+00,0.000000,0,0.000000,0.600021,0,0,0,0
8288309,607202,TCP,192.168.1.106,47940.0,52.87.241.159,443.0,156.0,ack,2549.0,64.0,...,1.475330e+09,1.475413e+09,82797.735474,17729288,214.127692,0.823887,78173,78116,12386218,5343070


In [9]:
# Count the number of rows
num_rows = df_merged.shape[0]

# Print the number of rows
print(f"The DataFrame has {num_rows} rows.")

The DataFrame has 8288310 rows.


In [10]:
# Read the mapping DataFrame from Excel
mapping_df = pd.read_excel('/gdrive/MyDrive/IomT/unsw_data/packet/mapping.xlsx', engine='openpyxl')

In [11]:
# Create a mapping dictionary
mapping_dict = dict(zip(mapping_df.packet_eth_src, mapping_df.target))

In [12]:
# Apply the mapping to the 'packet_eth_src' column in df_merged
df_merged['target'] = df_merged['packet_eth_src'].map(mapping_dict)

# Fill any NaN values with 'unknown'
df_merged['target'] = df_merged['target'].fillna('unknown')

In [13]:
# Count the number of packets for each target and print it
target_counts = df_merged['target'].value_counts()
print("Number of packets for each target:")
print(target_counts)

Number of packets for each target:
unknown               6214263
samsung_cam            394107
amazon_echo            374654
belkin_motion          337982
belkin_switch          265603
tp-link_cam            161748
netatmo_cam            160954
smart_things           121100
withings_sleep          87294
triby_speaker           36834
netatmo_weather         36063
pixstar_photoframe      26119
hp_printer              22698
lifx                    21632
ihome_plug              13540
tp-link_plug            11211
withings_scale           1387
nest_smoke               1040
blipcare_bp_meter          81
Name: target, dtype: int64


In [ ]:
# Assuming df_merged is your DataFrame
# First, filter targets with at least 10,000 packets
filtered_df = df_merged[df_merged['target'].map(df_merged['target'].value_counts() >= 26000)]

# Then, sample 10,000 packets per target
sampled_df = filtered_df.groupby('target').apply(lambda x: x.sample(n=26000, random_state=1)).reset_index(drop=True)

# Now, sampled_df will have 10,000 packets for each target that originally had at least 10,000 packets

In [ ]:
# Count the number of packets for each target and print it
target_counts_2 = sampled_df['target'].value_counts()
print("Number of packets for each target:")
print(target_counts_2)

Number of packets for each target:
amazon_echo           11000
belkin_motion         11000
belkin_switch         11000
netatmo_cam           11000
netatmo_weather       11000
pixstar_photoframe    11000
samsung_cam           11000
smart_things          11000
tp-link_cam           11000
triby_speaker         11000
unknown               11000
Name: target, dtype: int64


In [ ]:
# Assuming df is your DataFrame

def split_group(group):
    # Randomly sample 1000 rows for the test set
    test = group.sample(n=1000, random_state=1)
    # Use drop to remove the rows that were selected for the test set from the group to form the holdout set
    holdout = group.drop(test.index)
    return test, holdout

# Initialize empty DataFrames for the test and holdout sets
df_test = pd.DataFrame()
df_holdout = pd.DataFrame()

# Split each group and append to df_test and df_holdout
for name, group in sampled_df.groupby('target'):
    # Only split groups that have exactly 1100 flows
    if len(group) == 1100:
        test, holdout = split_group(group)
        df_test = df_test.append(test)
        df_holdout = df_holdout.append(holdout)

# Reset the index of df_test and df_holdout
df_test.reset_index(drop=True, inplace=True)
df_holdout.reset_index(drop=True, inplace=True)

# Now, df_test contains 1000 packets per target and df_holdout contains 100 packets per target for those targets with exactly 1100 flows,
# with no overlapping packets between them


In [ ]:
# Count the number of packets for each target and print it
target_counts_2 = df_test['target'].value_counts()
print("Number of packets for each target:")
print(target_counts_2)

Number of packets for each target:
amazon_echo           10000
belkin_motion         10000
belkin_switch         10000
netatmo_cam           10000
netatmo_weather       10000
pixstar_photoframe    10000
samsung_cam           10000
smart_things          10000
tp-link_cam           10000
triby_speaker         10000
unknown               10000
Name: target, dtype: int64


In [ ]:
# Count the number of packets for each target and print it
target_counts_2 = df_holdout['target'].value_counts()
print("Number of packets for each target:")
print(target_counts_2)

Number of packets for each target:
amazon_echo           1000
belkin_motion         1000
belkin_switch         1000
netatmo_cam           1000
netatmo_weather       1000
pixstar_photoframe    1000
samsung_cam           1000
smart_things          1000
tp-link_cam           1000
triby_speaker         1000
unknown               1000
Name: target, dtype: int64


In [ ]:
df_test.to_excel('/gdrive/MyDrive/IomT/unsw_data/packet/def_packet_data/df_train_test_packet_23_26.xlsx.xlsx', index=False)

In [ ]:
df_holdout.to_excel('/gdrive/MyDrive/IomT/unsw_data/packet/def_packet_data/df_holdout_packet_23_26.xlsx.xlsx', index=False)

In [ ]:
# Set Pandas to display all columns and rows
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)